**Model_2: a CNN model building**

In [1]:
import torchvision
from torchvision import datasets
from torchvision.transforms import ToTensor

train_data = datasets.FashionMNIST(
    root='data',
    train=True,
    download=False,
    transform=ToTensor(),
    target_transform=None
)


test_data = datasets.FashionMNIST(
    root='data',
    train=False,
    transform=ToTensor(),
    download=False
)

In [2]:
image,labels = train_data[0]

In [3]:
class_names=train_data.classes
print(class_names)

['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']


In [ ]:
# making into dataloader
from torch.utils.data import DataLoader
train_dataloader=DataLoader(dataset=train_data,batch_size=32,shuffle=True)
test_dataloader=DataLoader(dataset=test_data,batch_size=32, shuffle=False)


In [6]:
print(len(train_dataloader))
print(len(test_dataloader))

1875
313


In [9]:
train_feature_batch, train_label_batch=next(iter(train_dataloader))
print(train_label_batch)
print(train_feature_batch.shape)

tensor([9, 1, 8, 6, 6, 1, 4, 3, 9, 1, 9, 1, 2, 4, 6, 6, 4, 2, 4, 7, 5, 1, 0, 3,
        1, 2, 0, 5, 1, 9, 3, 8])
torch.Size([32, 1, 28, 28])


In [ ]:
# setting up agonistic device
import torch
device='cuda' if torch.cuda.is_available() else 'cpu'
print(device)


cpu


The structure of CNN follows
`Input layer` -> `[convolutional layer -> activation layer -> pooling layer]` -> `output layer`

`[convolutional layer -> activation layer -> pooling layer]` can be upscaled and repeated multiple times in CNN

This table serves as a general guideline for choosing an appropriate model architecture based on the type of data being processed.

| Problem Type | Recommended Model (Generally) | Example Library/Package |
| :--- | :--- | :--- |
| **Structured Data** | **Gradient Boosted Models**, **Random Forests**, XGBoost, LightGBM | `sklearn.ensemble`, `XGBoost` library |
| (e.g., spreadsheets, row/column data) | | |
| **Unstructured Data** | **Convolutional Neural Networks (CNNs)**, **Transformers** (e.g., for language/vision) | `torchvision.models`, `HuggingFace Transformers` |
| (e.g., Images, Audio, Free-form Text) | | |

In [ ]:
# for making CNN, we've `nn.Conv2d()` and `nn.MaxPool2d()`
from torch import nn

class FashionMNISTModelV2(nn.Module):
    def __init__(self, input_layer:int, hidden_layer:int, output_layer:int):
        super().__init__()
        self.block_1=nn.Sequential(
            nn.Conv2d(in_channels=input_layer,out_channels=hidden_layer, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_layer, out_channels=hidden_layer, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.block_2=nn.Sequential(
            nn.Conv2d(in_channels=hidden_layer, out_channels=hidden_layer, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_layer, out_channels=hidden_layer,
                      kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.classifier=nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=hidden_layer*7*7,out_features=output_layer)
        )

    def forward(self, x):
        x=self.block_1(x)
        x=self.block_2(x)
        x=self.classifier(x)
        return x
    
model_2=FashionMNISTModelV2(input_layer=1,output_layer=len(class_names),hidden_layer=10).to(device)
model_2


FashionMNISTModelV2(
  (block_1): Sequential(
    (0): Conv2d(1, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (block_2): Sequential(
    (0): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=490, out_features=10, bias=True)
  )
)

`in_channels (int)`:Number of channels in the input image.

`out_channels (int)`: Number of channels produced by the convolution.

`kernel_size (int/tuple)`:  Size of the convolving kernel/filter.

`stride (int/tuple/optional)`: or tuple, optional) - How big of a step the convolving kernel takes at a time. Default: 1.

`padding (int/tuple/str)`:Padding added to all four sides of input. Default: 0.

In [15]:
# loss and optimizer
loss_fn=nn.CrossEntropyLoss()
optimizer=torch.optim.SGD(params=model_2.parameters(),lr=0.1)


In [16]:
# accuracy fn
from helper_functions import accuracy_fn

*model_2: training loop*

In [17]:
def training_loop(model: nn.Module,
                  data_loader: torch.utils.data.DataLoader,
                  loss_fn: nn.modules,
                  accuracy_fn,
                  optimizer: torch.optim.Optimizer,
                  device: torch.device = device):
    train_loss = 0
    train_acc = 0
    for batch, (X, y) in enumerate(data_loader):

        # into device CPU
        X = X.to(device)
        y = y.to(device)

        model.train()  # train
        y_pred = model(X)  # forward pass
        loss = loss_fn(y_pred, y)  # loss
        train_loss += loss  # train loss
        train_acc += accuracy_fn(y, y_pred.argmax(dim=1))  # train accuracy
        optimizer.zero_grad()  # zero grad
        loss.backward()  # backward propagation
        optimizer.step()  # updation
    train_loss /= len(data_loader)
    train_acc /= len(data_loader)
    print(f"Train loss: {train_loss:.5f} | Train accuracy: {train_acc:.2f}%")

*model_2: testing_loop*

In [18]:
def testing_loop(model: nn.Module,
                 data_loader: torch.utils.data.DataLoader,
                 loss_fn: nn.Module,
                 accuracy_fn,
                 device: torch.device = device
                 ):
    test_loss, test_acc = 0, 0
    model.eval()
    with torch.inference_mode():
        for X, y in data_loader:
            X = X.to(device)
            y = y.to(device)
            test_pred = model(X)  # forward pass
            test_loss += loss_fn(test_pred, y)
            test_acc += accuracy_fn(y, test_pred.argmax(dim=1))
        test_loss /= len(data_loader)
        test_acc /= len(data_loader)
    print(f"Test loss: {test_loss:.5f} | Test accuracy: {test_acc:.2f}%\n")

*Training Testing Model*

In [19]:
torch.manual_seed(42)
epochs=3
for epoch in range(epochs):
        print(f"Epoch: {epoch}\n---------")
        training_loop(model=model_2,data_loader=train_dataloader,loss_fn=loss_fn,accuracy_fn=accuracy_fn,optimizer=optimizer,device=device)
        testing_loop(model=model_2,data_loader=test_dataloader,loss_fn=loss_fn,accuracy_fn=accuracy_fn,device=device)

        

Epoch: 0
---------
Train loss: 0.53410 | Train accuracy: 80.78%
Test loss: 0.38076 | Test accuracy: 86.33%

Epoch: 1
---------
Train loss: 0.34863 | Train accuracy: 87.51%
Test loss: 0.33510 | Test accuracy: 87.91%

Epoch: 2
---------
Train loss: 0.30969 | Train accuracy: 88.92%
Test loss: 0.31029 | Test accuracy: 88.84%



In [20]:
# model evaluation of model_2
torch.manual_seed(42)


def model_evaluation(model: torch.nn.Module,
                     data_loader: torch.utils.data.DataLoader,
                     loss_fn: torch.nn.Module,
                     accuracy_fn):

    loss = 0
    acc = 0
    model.eval()
    with torch.inference_mode():
        for X, y in data_loader:
            model_pred = model(X)
            loss += loss_fn(model_pred, y)
            acc += accuracy_fn(y_true=y, y_pred=model_pred.argmax(dim=1))
        loss /= len(data_loader)
        acc /= len(data_loader)

    return {"model_name": model.__class__.__name__,  # only works when model was created with a class
            "model_loss": loss.item(),
            "model_acc": acc}

In [22]:
model_2_performance=model_evaluation(model=model_2,data_loader=test_dataloader,loss_fn=loss_fn,accuracy_fn=accuracy_fn)
print(model_2_performance)


{'model_name': 'FashionMNISTModelV2', 'model_loss': 0.31028592586517334, 'model_acc': 88.83785942492013}
